In [3]:
from thermo import ChemicalConstantsPackage
from thermo.unifac import UNIFAC, DOUFSG, DOUFIP2016
from chemicals import search_chemical
import numpy as np
import pandas as pd
from IPython.display import clear_output
import sys
sys.path.insert(0,'/home/garren/HPC Files')
from All_code import subsets

In [2]:
with pd.ExcelFile('/home/garren/Documents/MEng/Code/Latest_results/HPC Files/TestingData_Final.xlsx') as f:
    df = pd.read_excel(f)

In [ ]:
y = []
IUPAC_mix = []
all_chem_mix = []
for i in range(len(df)):
    print(f'Datapoint {i+1} out of {len(df)}')
    c1 = df['Component 1'][i]
    c2 = df['Component 2'][i]
    IUPAC_mix += [[c1, c2]]
    constants,_ = ChemicalConstantsPackage.from_IDs(IUPAC_mix[i])
    all_chem_mix += [constants.UNIFAC_Dortmund_groups]
    x = df['Composition component 1 [mol/mol]'][i]
    xs = [x, 1-x]
    T = df['Temperature [K]'][i]
    GE = UNIFAC.from_subgroups(T=T, xs=xs, chemgroups=all_chem_mix[i], version=1, interaction_data=DOUFIP2016, subgroups=DOUFSG)
    y += [GE.HE()]
    clear_output(wait=True)

In [ ]:
data = {'Component 1': np.array(IUPAC_mix)[:,0],
        'Component 2': np.array(IUPAC_mix)[:,1],
        'UNIFAC_DMD [J/mol]': y}
data = pd.DataFrame(data)
data.to_excel('Thermo_UNIFAC_DMD.xlsx', index=False, header=True, sheet_name='UNIFAC_DMD')

In [ ]:
copy_df = df.copy()
copy_df['UNIFAC_DMD [J/mol]'] = y

In [ ]:
copy_df.to_excel('/home/garren/Documents/MEng/Code/Latest_results/HPC Files/TestingData_Final.xlsx', index=False, header=True)

In [ ]:
subset_df, subset_Indices, subset_Indices_T, Info_Indices, init_indices, init_indices_T = subsets(np.array(['all'])).get_subset_df()

In [ ]:
N = len(Info_Indices['Component names']['Index'])
Idx_known = subset_df.iloc[subset_Indices_T[:,0],7:9].to_numpy()
Idx_unknown = np.array([[i, j] for i in range(N) for j in range(i+1,N)])
idx = np.sum(np.char.add(Idx_unknown[:,0].astype(str), Idx_unknown[:,1].astype(str))[:,np.newaxis] ==
             np.char.add(Idx_known[:,0].astype(str), Idx_known[:,1].astype(str))[np.newaxis,:], axis=1) == 0
Idx_unknown = Idx_unknown[idx,:]

In [ ]:
T2 = np.array([288.15, 298.15, 308.15])
x2 = np.linspace(0,1, 21)[1:-1]

In [ ]:
dic = {'Component 1': [], 'Component 2': [],
        'Composition component 1 [mol/mol]': [], 'Temperature [K]': [], 
        'UNIFAC_DMD [J/mol]': []}
        
for i in range(Idx_unknown.shape[0]):
    print(f'Datapoint {i+1} out of {Idx_unknown.shape[0]}')
    c1 = Info_Indices['Component names']['IUPAC'][Idx_unknown[i,0]]
    c2 = Info_Indices['Component names']['IUPAC'][Idx_unknown[i,1]]
    IUPAC_mix = [c1, c2]
    constants,_ = ChemicalConstantsPackage.from_IDs(IUPAC_mix)
    all_chem_mix = constants.UNIFAC_Dortmund_groups
    for j in range(len(T2)):
        for k in range(len(x2)):
            dic['Composition component 1 [mol/mol]'] += [x2[k]]
            dic['Temperature [K]'] += [T2[j]]
            dic['Component 1'] += [c1]
            dic['Component 2'] += [c2]
            xs = [x2[k], 1-x2[k]]
            GE = UNIFAC.from_subgroups(T=T2[j], xs=xs, chemgroups=all_chem_mix, version=1, interaction_data=DOUFIP2016, subgroups=DOUFSG)
            dic['UNIFAC_DMD [J/mol]'] += [GE.HE()]
    clear_output(wait=True)

In [ ]:
pd.DataFrame(dic).to_excel('Thermo_UNIFAC_DMD_unknown.xlsx', index=False, header=True, sheet_name='UNIFAC_DMD_unknown')

In [ ]:
with pd.ExcelFile('/home/garren/Documents/MEng/Code/Latest_results/Alkane_Alcohol_Subset_of_Testing_Data_and_UNIFAC_obtained_from_Aspen.xlsx') as f:
    df = pd.read_excel(f, sheet_name='TestingData')

In [ ]:
y = []

for i in range(len(df)):
    print(f'Datapoint {i+1} out of {len(df)}')
    c1 = df['Component 1'][i]
    c2 = df['Component 2'][i]
    IUPAC_mix = [c1, c2]
    constants,_ = ChemicalConstantsPackage.from_IDs(IUPAC_mix)
    all_chem_mix = constants.UNIFAC_Dortmund_groups
    x = df['Composition component 1 [mol/mol]'][i]
    xs = [x, 1-x]
    T = df['Temperature [K]'][i]
    GE = UNIFAC.from_subgroups(T=T, xs=xs, chemgroups=all_chem_mix, version=1, interaction_data=DOUFIP2016, subgroups=DOUFSG)
    y += [GE.HE()]
    clear_output(wait=True)


In [ ]:
pd.DataFrame({'UNIFAC_DMD [J/mol]': y}).to_excel('/home/garren/Documents/MEng/Code/Latest_results/Thermo_UNIFAC_DMD_TestingData.xlsx', index=False, header=True, sheet_name='UNIFAC_DMD_TestingData')

In [7]:
with pd.ExcelFile('/home/garren/Documents/MEng/Code/Latest_results/HPC Files/TestingData_Final.xlsx') as f:
    df = pd.read_excel(f)

In [8]:
all_mix, idx = np.unique(np.char.add(np.char.add(df['Component 1'].to_numpy().astype(str), ' + '),
                                df['Component 2'].to_numpy().astype(str)), return_index=True)
all_mix = all_mix[np.argsort(idx)]
idx = np.sort(idx)
c1 = df['Component 1'].to_numpy()[idx].astype(str)
c2 = df['Component 2'].to_numpy()[idx].astype(str)
x_plot = np.linspace(0,1,101)

testing_dic = {'Component 1': [],
               'Component 2': [],
               'Temperature [K]': [],
               'Composition component 1 [mol/mol]': [],
               'UNIFAC_DMD [J/mol]': []}
for i in range(c1.shape[0]):
    print(f'{i+1} out of {c1.shape[0]} mixtures evaluated')
    IUPAC_mix = [c1[i], c2[i]]
    constants,_ = ChemicalConstantsPackage.from_IDs(IUPAC_mix)
    all_chem_mix = constants.UNIFAC_Dortmund_groups
    
    idx1 = df['Component 1'].to_numpy() == c1[i]
    idx2 = df['Component 2'].to_numpy() == c2[i]
    idx = (idx1.astype(int) + idx2.astype(int)) == 2

    T_mix = df['Temperature [K]'].to_numpy()[idx]
    T_mix_unique = np.unique(T_mix)
    T_mix_unique = np.concatenate([T_mix_unique.astype(int)+0.15, T_mix_unique.astype(int)+1.15])
    T_mix_unique = np.unique(T_mix_unique)
    T_eval = T_mix_unique[np.sum(np.abs(T_mix_unique[:,np.newaxis]-T_mix[np.newaxis,:])<=0.5, axis=1) > 0]
    for T in T_eval:
        for x in x_plot:
            xs = [x, 1-x]
            GE = UNIFAC.from_subgroups(T=T, xs=xs, chemgroups=all_chem_mix, version=1, interaction_data=DOUFIP2016, subgroups=DOUFSG)
            testing_dic['Component 1'] += [c1[i]]
            testing_dic['Component 2'] += [c2[i]]
            testing_dic['Temperature [K]'] += [T]
            testing_dic['Composition component 1 [mol/mol]'] += [x]
            testing_dic['UNIFAC_DMD [J/mol]'] += [GE.HE()]
    clear_output(wait=True)

212 out of 212 mixtures evaluated


In [9]:
with pd.ExcelFile('/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Sorted Data.xlsx') as f:
    df = pd.read_excel(f)
    Indices_T = pd.read_excel(f, sheet_name='Indices_T')
Indices_T = Indices_T.to_numpy()[:, :2]

In [10]:
all_mix, idx = np.unique(np.char.add(np.char.add(df['Component 1'].to_numpy()[Indices_T[:,0]].astype(str), ' + '),
                                df['Component 2'].to_numpy()[Indices_T[:,0]].astype(str)), return_index=True)
all_mix = all_mix[np.argsort(idx)]
idx = np.sort(idx)
c1 = df['Component 1'].to_numpy()[Indices_T[:,0]][idx].astype(str)
c2 = df['Component 2'].to_numpy()[Indices_T[:,0]][idx].astype(str)
x_plot = np.linspace(0,1,101)

training_dic = {'Component 1': [],
               'Component 2': [],
               'Temperature [K]': [],
               'Composition component 1 [mol/mol]': [],
               'UNIFAC_DMD [J/mol]': []}
for i in range(c1.shape[0]):
    print(f'{i+1} out of {c1.shape[0]} mixtures evaluated')
    IUPAC_mix = [c1[i], c2[i]]
    constants,_ = ChemicalConstantsPackage.from_IDs(IUPAC_mix)
    all_chem_mix = constants.UNIFAC_Dortmund_groups
    
    idx1 = df['Component 1'].to_numpy() == c1[i]
    idx2 = df['Component 2'].to_numpy() == c2[i]
    idx = (idx1.astype(int) + idx2.astype(int)) == 2

    T_mix = df['Temperature [K]'].to_numpy()[idx]
    T_mix_unique = np.unique(T_mix)
    T_mix_unique = np.concatenate([T_mix_unique.astype(int)+0.15, T_mix_unique.astype(int)+1.15])
    T_mix_unique = np.unique(T_mix_unique)
    T_eval = T_mix_unique[np.sum(np.abs(T_mix_unique[:,np.newaxis]-T_mix[np.newaxis,:])<=0.5, axis=1) > 0]
    for T in T_eval:
        for x in x_plot:
            xs = [x, 1-x]
            GE = UNIFAC.from_subgroups(T=T, xs=xs, chemgroups=all_chem_mix, version=1, interaction_data=DOUFIP2016, subgroups=DOUFSG)
            training_dic['Component 1'] += [c1[i]]
            training_dic['Component 2'] += [c2[i]]
            training_dic['Temperature [K]'] += [T]
            training_dic['Composition component 1 [mol/mol]'] += [x]
            training_dic['UNIFAC_DMD [J/mol]'] += [GE.HE()]
    clear_output(wait=True)

781 out of 781 mixtures evaluated


In [11]:
with pd.ExcelWriter('/home/garren/Documents/MEng/Code/Latest_results/HPC Files/UNIFAC_Plots.xlsx') as writer:
    pd.DataFrame(training_dic).to_excel(writer, index=False, header=True, sheet_name='Training_Plots')
    pd.DataFrame(testing_dic).to_excel(writer, index=False, header=True, sheet_name='Testing_Plots')